In [ ]:
import os
import re
import json
import chardet
import scr.public_function as pf

# NOTE 中间的所有文件都是UTF8
encoding = 'cp932'
output_encoding = 'utf16'

pf._init_()
name = ''

In [ ]:
# 从游戏原文本文件提取日文

def get_scenario_from_origin(data: list) -> list:
    text_all = []
    cnt = 0
    # START FIXME
    for line in data:
        if pf.has_jp(line) and line[0] not in "#*[":
            text_all.append(line)
        cnt+=1
    # END
    return text_all

pf.extract_jp(get_scenario_from_origin, encoding)


In [10]:
pf.Lilim.output_hook_dict()

添加条目:  14297


In [ ]:
# 创建翻译字典
pf.create_dict(lambda x:x[:-1])

In [ ]:
# 检查字典key数量是否正确
pf.check_dict()

In [ ]:
# 把字典翻译

# print(format_yu_ris('歩「じゃあ≪射精／だ≫して……ね？　いっぱい……≪射精／だ≫してほしい」'))
pf.translate(delete_func=None)
# print(pf._translate('「あーうそうそ！　冗談だってば！\r\n　昼飯は後でちゃんと喰わせるから、\r\n　今は俺を信じてもうちょっと歩こうぜ！」', ))

In [ ]:
# 检查是否还有没翻译的语句
pf.check_dict_untranslated()

In [ ]:
# 构建对比
pf.create_contrast()

In [ ]:
# 替换文本
def find_all(data:list, failed_text:list, jp_chs:dict):
    record = 0
    cnt = 0                         # 当前行
    start = False
    for line in data:
        #START FIXME 替换目标文本
        if pf.has_jp(line) and line[0] not in "#*[":
            key = line[:-1]
            if key in jp_chs and jp_chs[key]:
                record += 1
                data[cnt] = data[cnt].replace(key, jp_chs[key])
            else:
                failed_text.append(key+'\n')
        cnt += 1
        # END 
    return record

pf.replace_all(find_all, encoding, output_encoding)

In [5]:
a = '54 70 57 6F 72 64 31 30 35 00 00 00 00 01 00 00 00 03 00 00 00 00 00 00 00 00 00 00 00 03 00 00 00 06 00 00 00 00 0B 00 00 00 4D 45 4E 55 45 4E 41 42 4C 45 44 6 00 00 00 00 01 00 00 00 01 06 00 00 00 00 06 00 00 00 4D 45 53 45 4E 44 54 70 57 6F 72 64 31 30 35 00 00 00 00 01 00 00 00 03 00 00 00 00 00 00 00 00 00 00 00 03 00 00 00 06 00 00 00 00 0B 00 00 00 4D 45 4E 55 45 4E 41 42 4C 45 44 06 00 00 00 00 01 00 00 00 01 06 00 00 00 00 06 00 00 00 4D 45 53 45 4E 44'
a = a.split()
a = '\\x'+'\\x'.join(a)
a = eval(a)
print(a)

SyntaxError: unexpected character after line continuation character (<string>, line 1)

In [9]:
b'\x81\x63'.decode('cp932')

'…'

In [16]:
a = 0
a += 1 if 5 < 100 else 4
print(a)

1


In [1]:
a = [1,3,4,5,6]
for i, c in enumerate(a):
    print(i, c)

0 1
1 3
2 4
3 5
4 6
